## Welcome

To select origin location left click 
To select destination location right-click



In [1]:
# Import libraries
import os

# for mapping and creating widgets
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import Label
import folium

# open source routing 

import openrouteservice as ors


In [2]:
# Create an interactive map
Map = geemap.Map(center=[18.25, -66.25], zoom=9, add_google_map=False)
Map.add_basemap('Esri National Geographic') # nat geo basemap


# pulling in resources layers
available_csv= 'https://raw.githubusercontent.com/cotag11/puerto-rico-files/main/available.csv'
needed_csv= 'https://raw.githubusercontent.com/cotag11/puerto-rico-files/main/needed.csv'
#obstructions_csv= 'https://raw.githubusercontent.com/cotag11/puerto-rico-files/main/obstructions.csv'

# converting data to ee objects
available = geemap.xy_to_points(available_csv, latitude='y', longitude='x')
needed = geemap.xy_to_points(needed_csv, latitude='y', longitude='x')
#obstructions = geemap.xy_to_points(obstructions_csv, latitude='y', longitude='x')

Map.addLayer(available, {'color': 'green'}, 'Available Resources')
Map.addLayer(needed, {'color': 'red'}, 'Needed Resources')
#Map.addLayer(obstructions, {'color': 'black'}, 'Roads Blocked available')Map.addLayerControl()
Map

Data downloaded to: C:\Users\Gizelle\Downloads\available.csv
Data downloaded to: C:\Users\Gizelle\Downloads\needed.csv


Map(center=[18.25, -66.25], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

In [3]:
style = {'description_width': 'initial'}

# print statements are visible through this widget
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

# numeric widgets
origin_lat_widget= widgets.FloatText(value=0.00, description='Origin Lat:', disabled=False)
origin_lon_widget= widgets.FloatText(value=-0.0,description='Origin Long:',disabled=False)
destin_lat_widget= widgets.FloatText(value=0.0, description='Dest. Lat:',  disabled=False)
destin_lon_widget= widgets.FloatText(value=-0.00,description='Dest. Long:', disabled=False)

# 'submit' button
submit = widgets.Button(description='Calculate route', button_style='primary', tooltip='Click me', style=style)

download_widget = widgets.Checkbox(value=False,description='Download Route directions',style=style)

# combined widgets
full_widget = widgets.VBox([  # vertical box
    widgets.HBox([origin_lat_widget, origin_lon_widget, download_widget]), # horizontal box
    widgets.HBox([destin_lat_widget, destin_lon_widget, submit]) # horizontal box
])

full_widget

In [13]:
# get origin and destination values through user interaction

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
      # clears previous messages
    
    if kwargs.get('type') == 'click' : # specify left click  for origin location
        Map.default_style = {'cursor': 'wait'} 
        output_widget.clear_output()
        # inverses the lat long postions
        xy = ee.Geometry.Point(latlon[::-1]) 
        
        try:
            # puts numeric values in the widget based on where it was clicked
            origin_lat_widget.value = float(latlon[0]) 
            origin_lon_widget.value = float(latlon[1])
            
            # removes destination location- by specifying a value of 0
            destin_lat_widget.value= float(0.00)
            destin_lon_widget.value= float(0.00)
            
            # only the first five layers are displayed
            Map.layers = Map.layers[:4]
            
            # 'origin' location is plotted
            Map.addLayer(xy, {'color':'yellow', 'size':'7'}) 
        
        # Suggestion: filter locations such that they are within Puerto Rico (optional), not within water, and close to roads
        
        except:
            print('No feature could be found')
            Map.layers = Map.layers[:4]    
     
        Map.default_style = {'cursor': 'pointer'}
        
    elif kwargs.get('type') == 'contextmenu': # specify right click for destination locations
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        
        try: 
            # puts numeric values in the widget based on where it was clicked
            destin_lat_widget.value= float(latlon[0])
            destin_lon_widget.value= float(latlon[1])
            
            Map.layers = Map.layers[:5]
            
            # 'destination' location is plotted 
            Map.addLayer(xy, {'color':'blue', 'size':'7'}) 
            
        # Suggestion: filter locations such that they are within Puerto Rico (optional), not within water, and close to roads   
        except: 
            print('No feature could be found')
            Map.layers = Map.layers[:5]   

        Map.default_style = {'cursor': 'pointer'}
  

    else:
        Map.draw_count = 0

Map.on_interaction(handle_interaction)

In [9]:
# defines what happens when the submit button is clicked

def submit_clicked(b):
      with output_widget:
        output_widget.clear_output()

        Map.default_style = {'cursor': 'wait'}
        try:
            # get lat long values for origin and destination 
            origin_lat_id = origin_lat_widget.value
            origin_lon_id = origin_lon_widget.value
            destin_lat_id = destin_lat_widget.value
            destin_lon_id = destin_lon_widget.value

            # calculate route here
            client = ors.Client(key='5b3ce3597851110001cf6248e5263feb3dc247cfb7b2d52fe7725e10') # 2000 quota for 24 hours
            # link to create your own key/ keys : https://openrouteservice.org/dev/#/login 
            
            coordinates = [[origin_lon_id, origin_lat_id], [destin_lon_id, destin_lat_id]] # coordinates order is [lon, lat]
            route = client.directions(coordinates=coordinates, profile='driving-car',  format='geojson')
            # profile options: driving-car, driving-hgv, foot-walking, foot-hiking, cycling-regular, cycling-road, cycling-mountain, cycling-electric
            # the open source package throws an error if asked to compute distance between two points that aren't close to roads
            
            route_json= geemap.geojson_to_ee(route) # geemap requires it to be in ee.Geometry format
          
            Map.addLayer(route_json)  
            
            # Travel distance (miles) and time (minutes)
            print('Total Distance:', round(route['features'][0]['properties']['segments'][0]['distance']*0.000621371, 2), 'miles', 
                  'Total Time:', round(route['features'][0]['properties']['segments'][0]['duration']*0.016666667, 2),'mins')

            download = download_widget.value # checks if download widget is clicked
            
            # downloads / prints directions
            if download: 
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_name = 'Directions_' + geemap.random_string() + '.txt'
                out_file = os.path.join(out_dir, out_name)

                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                with open(out_file, 'w') as f:
                    f.write('Directions: \n')
                    for index, i in enumerate(route['features'][0]['properties']['segments'][0]['steps']):
                        if index==0:
                            f.write(('{} for {} miles \n').format(i['instruction'], round(i['distance']*0.000621371, 2)))
                        elif index ==1: 
                            f.write(('{} \n'.format(i['instruction'])))
                        else:
                            f.write(('In {} miles, {} \n'.format(
                                round(route['features'][0]['properties']['segments'][0]['steps'][index-1]['distance']*0.000621371, 3), 
                                i['instruction'])))

                link = geemap.create_download_link(out_file, title="Click here to download the directions: ")
                display(link)
                
            else:
                print('Directions: ')
                for index, i in enumerate(route['features'][0]['properties']['segments'][0]['steps']):
                    if index==0:
                        print(('{} for {} miles').format(i['instruction'], round(i['distance']*0.000621371, 2)))
                    elif index ==1: 
                        print(i['instruction'])
                    else:
                        print(('In {} miles, {}'.format(
                            round(route['features'][0]['properties']['segments'][0]['steps'][index-1]['distance']*0.000621371, 3), 
                            i['instruction'])))
                
        except Exception as e:
            print(e)
            print('An error occurred during computation.')        

        Map.default_style = {'cursor': 'default'}


submit.on_click(submit_clicked)